# Model Selection

In [1]:
import pandas as pd
import numpy as np
import glob, sys, os
sys.path.append('..')

In [2]:
from modules.plotting_metrics import PlotMetric
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='white', context='talk', font_scale=0.8)

In [3]:
file_name = './df_DkSc_results_COCRYS_DEKOIS_DUD.pkl'
X_merged_dksc = pd.read_pickle(file_name)
# Extract activity column
y_true_merged = X_merged_dksc['activity']
# Drop column from merged_dkksc
X_merged_dksc = X_merged_dksc.drop('activity', axis=1)
X_merged_dksc.shape
y_true_merged.loc['DEKOIS'].sum()

40

### Timeout Decorator

In [4]:
import signal
from functools import wraps

def timeout(n_seconds=300):
    '''Stops a function execution after n seconds'''
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            # Set alarm for n seconds
            signal.alarm(n_seconds)
            try:
                # Call decorated func
                return func(*args, **kwargs)
#             except TimeoutError as e:
                print(f'Execution finished after {n_seconds}:', e)
            finally:
                # Cancel Alarm
                signal.alarm(0)
        return wrapper
    return decorator

## Scaffold Splitting

In [5]:
#*************************************************
# Functions to compute stratify scaffold splitting
#*************************************************
from scaffold_splitter import train_test_scaffold_split

In [6]:
# Compute or load the dataframe containing the Generic Murcko Scaffolds
file = './df_COCRYS_DUD_DEKOIS_Murcko_Scaffolds_SMILES.obj'

df_scff_murcko = pd.read_pickle(file)

## Train/test on the same dataset 

### Learning Curves

In [7]:
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import learning_curve

In [8]:
def plot_learning_curves(estimator, X, y, title, ylim=[0.5,1], axes=None,
                         cv=3, train_sizes=np.linspace(0.1, 1.0, 10), 
                         scoring='roc_auc', n_jobs=4):
    '''
    Plot estimator performance on the training and validation
    sets as a function of the training set size.

    Parameters
    ----------
    estimator: sklearn estimator object type
       Object type that implements the "fit" and "predict method"

    X: array-like, shape (m_samples, n_features)
        Training array with m_samples and n_features.
    y: array-like, shape (m_samples)
        Target array relative to X with m labels.
    axes: array of 3 axes
        matplotlib axes array to append the generated plot
    ylim: array
       
    '''
    if axes == None:
        _, axes = plt.subplots(1, 1, figsize=(5, 5))
        
    axes.set(title=title, ylim=ylim, xlabel='Training examples', ylabel=f'Metric: {scoring}')
    # Use learning_curve function from sklearn
    train_sizes, train_scores, test_scores, fit_times, _ = \
       learning_curve(estimator, X, y, scoring=scoring, return_times = True,
                      cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    # Compute useful metrics
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
#     fit_times_mean = np.mean(fit_times, axis=1)
#     fit_times_std = np.std(fit_times, axis=1)
    # Plot the learning curve
    axes.grid()
    axes.fill_between(train_sizes, train_scores_mean - train_scores_std,
                        train_scores_mean + train_scores_std, alpha=0.1, color='r')
    axes.fill_between(train_sizes, test_scores_mean - test_scores_std,
                        test_scores_mean + test_scores_std, alpha=0.1, color='g')
    axes.plot(train_sizes, train_scores_mean, 'o-', color='r',
                label='Train score')
    axes.plot(train_sizes, test_scores_mean, 'o-', color='g',
                label='Cross-validation score')
    axes.legend(loc='lower right')
    

### Hyperparameters tunning: Grid Search

In [9]:
def run_grid_search(estimator, X_train, y_train, X_test, y_test, hyperparams,  cv_value=5, 
                    scoring='roc_auc', splitting='random', std_scale=False):
    # Format the hyperparms
    hyperparams_dict = {'estimator__' + key: val for key, val in hyperparams.items()}
    
    if std_scale:
        # Create the Pipe
        scaler = StandardScaler()
        pipe = Pipeline([('scaler', scaler),
                         ('estimator', estimator)])
    else:
        pipe = estimator
    
    # Do Grid Search
    gs = GridSearchCV(estimator = pipe, param_grid = hyperparams,
                     cv = cv_value, scoring = scoring, n_jobs = 6, refit = True)

    # Train the model
    gs.fit(X_train, y_train) 

    # Predictions
    y_train_predict = gs.predict_proba(X_train)
    y_test_predict = gs.predict_proba(X_test)
    
    # Values to return
    n_train_mols    = y_train.shape[0]
    n_train_actives = y_train.sum()
    n_test_mols     = y_test.shape[0]
    n_test_actives  = y_test.sum()
    mean_cv_roc     = gs.best_score_
    train_roc       = roc_auc_score(y_train, y_train_predict[:, 1])
    test_roc        = roc_auc_score(y_test, y_test_predict[:, 1])
    best_params     = gs.best_params_

    # Print some values
    print(f'No. of molecules in train set: {n_train_mols}, with {n_train_actives} actives.')
    print(f'No. of molecules in test set: {n_test_mols}, with {n_test_actives} actives.')
    print('')
    print('*'*10, 'GRID SEARCH RESULTS', '*'*10)
    print('- Mean CV ROC-AUC:\t{:.3f}'.format(mean_cv_roc))
    print('- Train ROC-AUC:  \t{:.3f}'.format(train_roc))
    print('- Test ROC-AUC:   \t{:.3f}'.format(test_roc))
    print('- Best hyperparameters', best_params)
    print('**'*21)
    print('')
    
    return [n_train_mols, n_train_actives, n_test_mols, n_test_actives,
            mean_cv_roc, train_roc, test_roc, best_params]

### Function to report the Best Conformation's ROC-AUC for a given subset of samples

In [10]:
#************************************************************************
# Returns the best conformatio's ROC-AUC value of a given subset X and y
#************************************************************************

def get_roc_auc_DkSc(X_train, y_train, X_test, y_test, verbose=True):
    roc_auc_train = X_train.apply(
        lambda x: roc_auc_score(y_true= y_train, y_score= -x), axis=0)
    roc_auc_test = X_test.apply(
        lambda x: roc_auc_score(y_true= y_test, y_score= -x), axis=0)
    # Values to return
    train_best_roc = roc_auc_train.max()
    train_median   = roc_auc_train.median()
    train_mean     = roc_auc_train.mean()
    test_best_roc = roc_auc_test.max()
    test_median   = roc_auc_test.median()
    test_mean     = roc_auc_test.mean()
    
    if verbose:
        print("***** Best Conformation's ROC-AUC using docking scores *****")
        
        print("> Train best conf. ROC-AUC: {:.3f}".format(train_best_roc) +
              " \t> median: {:.3f}".format(train_median) +
              ', mean: {:.3f}'.format(train_mean))
        
        print("> Test best conf. ROC-AUC: {:.3f}".format(test_best_roc) +
              " \t> median: {:.3f}".format(test_median) +
              ', mean: {:.3f}'.format(test_mean))
        print('**'*32)
    # return a list of results to capture by the  wrapper function
    return [train_best_roc, train_median, train_mean,
            test_best_roc, test_median, test_mean]
        

In [11]:
#******************************************
# Decorator functions to capture GS results
#******************************************
from functools import wraps

def capture_GS_results(results_dict=None, capture=True):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            if results_dict != None and capture:
                results = func(*args, **kwargs)
                # Create a key with the first four values
                key = '_'.join(results[:4])
                # Append results to the results dictionary
                results_dict[key] = results
            else:
                return func(*args, **kwargs)
        return wrapper
    return decorator
            
#******************************************************
# Dictionary to capture GS results using the decorator 
#******************************************************
results_dict = {}

#******************************************************
# Function to Split and run Grid Search
#******************************************************
@capture_GS_results(results_dict)
def split_and_gs(train_name, test_name, estimator_name,
                 X, y, estimator, hyperparams, splitting='random', 
                 test_size=0.25, scaffold_series=None, **kwargs):
    '''Given a X and y sets, a sklean estimator and an splitting method, 
    performs Grid Search CV using the parsed hyperparams'''
    #**************
    # Do the split
    #**************
    if splitting == 'scaffold':
        X_train, X_test, y_train, y_test = \
            train_test_scaffold_split(X, y, scaffold_series = scaffold_series,
                test_size=test_size, stratify=y)
    elif splitting == 'random':
        X_train, X_test, y_train, y_test = \
            train_test_split(X, y, test_size=test_size, stratify=y)
        
    # Verbose
    print(f'{estimator_name} => Train: {train_name}; Test: {test_name}; split: {splitting}')
    
    # Function to run Grid Search
    #----------------------------
    gs_results = run_grid_search(estimator, 
                    X_train, y_train, X_test, y_test, 
                    hyperparams = hyperparams,  **kwargs)

    # Function to extract ROC results from DkSc values 
    #-------------------------------------------------
    dksc_results = get_roc_auc_DkSc(X_train, y_train, X_test, y_test)
    
    # Return both list of results to captured by the decorador function
    return [train_name, test_name, estimator_name, splitting] + gs_results + dksc_results

In [12]:
results_dict

{}

#  Hyperparameter Tunning: Grid Search
<h2 style='background-color: #F9E5AB; padding: 5px;'>
    Train-Test with DEKOIS Library
</h2>
<div style='background-color: #FE8460; min-height: 5px'></div>

In [13]:
#### library = 'DEKOIS'
library = 'DEKOIS'

# Train and test over DEKOIS
X = X_merged_dksc.loc[library]
y = y_true_merged.loc[library]
scaffold_series = df_scff_murcko['scff_generic'].loc[library]

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Linear SVM </h3>
<b>DEKOIS  - Random and Stratified Scaffold Splitting</b>

In [14]:
%%time
from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': [1]}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams, 
             splitting='random', test_size=0.25, 
             scaffold_series=None)

LinearSVC => Train: DEKOIS; Test: DEKOIS; split: random
No. of molecules in train set: 915, with 30 actives.
No. of molecules in test set: 306, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.829
- Train ROC-AUC:  	1.000
- Test ROC-AUC:   	0.886
- Best hyperparameters {'C': 1}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.923 	> median: 0.851, mean: 0.848
> Test best conf. ROC-AUC: 0.877 	> median: 0.785, mean: 0.781
****************************************************************
CPU times: user 718 ms, sys: 302 ms, total: 1.02 s
Wall time: 2.82 s


In [15]:
%%time
# from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-15, 1e-6, 4)}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
              X, y, estimator, hyperparams, 
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

LinearSVC => Train: DEKOIS; Test: DEKOIS; split: scaffold
No. of molecules in train set: 885, with 30 actives.
No. of molecules in test set: 336, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.847
- Train ROC-AUC:  	0.875
- Test ROC-AUC:   	0.956
- Best hyperparameters {'C': 1e-15}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.884 	> median: 0.816, mean: 0.812
> Test best conf. ROC-AUC: 0.965 	> median: 0.897, mean: 0.893
****************************************************************
CPU times: user 625 ms, sys: 62.1 ms, total: 687 ms
Wall time: 1.3 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Radial Basis Function SVM </h3>
<b>DEKOIS  - Random and Stratified Scaffold Splitting</b>

In [16]:
%%time
estimator_name = 'rbfSVC'
estimator = SVC(kernel = 'rbf', probability=True)
hyperparams = {'C': np.geomspace(1e-8, 1e-2, 4), 
               'gamma': np.geomspace(1e-6, 1e0, 4)}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams, 
             splitting='random', test_size=0.25, 
             scaffold_series=None)

rbfSVC => Train: DEKOIS; Test: DEKOIS; split: random
No. of molecules in train set: 915, with 30 actives.
No. of molecules in test set: 306, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.873
- Train ROC-AUC:  	0.875
- Test ROC-AUC:   	0.943
- Best hyperparameters {'C': 1e-08, 'gamma': 1e-06}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.888 	> median: 0.818, mean: 0.815
> Test best conf. ROC-AUC: 0.944 	> median: 0.880, mean: 0.878
****************************************************************
CPU times: user 931 ms, sys: 154 ms, total: 1.09 s
Wall time: 2.48 s


In [17]:
%%time
estimator_name = 'rbfSVC'
estimator = SVC(kernel = 'rbf', probability=True)
hyperparams = {'C': np.geomspace(1e-8, 1e-2, 4), 
               'gamma': np.geomspace(1e-6, 1e0, 4)}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams, 
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

rbfSVC => Train: DEKOIS; Test: DEKOIS; split: scaffold
No. of molecules in train set: 885, with 30 actives.
No. of molecules in test set: 336, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.846
- Train ROC-AUC:  	0.875
- Test ROC-AUC:   	0.953
- Best hyperparameters {'C': 1e-08, 'gamma': 1e-06}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.884 	> median: 0.816, mean: 0.812
> Test best conf. ROC-AUC: 0.965 	> median: 0.897, mean: 0.893
****************************************************************
CPU times: user 918 ms, sys: 136 ms, total: 1.05 s
Wall time: 2.36 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'> GS: Logistic Regression </h3>
<b>DEKOIS  - Random and Stratified Scaffold Splitting</b>

In [18]:
%%time
from sklearn.linear_model import LogisticRegression

estimator_name = 'LogReg'
estimator = LogisticRegression()
hyperparams = {'C': [0.00001, 0.0001, 0.001, 0.01, 0.1], 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams, 
             splitting='random', test_size=0.25, 
             scaffold_series=None)

LogReg => Train: DEKOIS; Test: DEKOIS; split: random


/home/joel/anaconda3/envs/mds/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


No. of molecules in train set: 915, with 30 actives.
No. of molecules in test set: 306, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.922
- Train ROC-AUC:  	0.950
- Test ROC-AUC:   	0.805
- Best hyperparameters {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.929 	> median: 0.867, mean: 0.860
> Test best conf. ROC-AUC: 0.857 	> median: 0.747, mean: 0.744
****************************************************************
CPU times: user 1.23 s, sys: 601 ms, total: 1.83 s
Wall time: 1.84 s


In [19]:
%%time
# from sklearn.linear_model import LogisticRegression

estimator_name = 'LogReg'
estimator = LogisticRegression()
hyperparams = {'C': [0.00001, 0.0001, 0.001, 0.01, 0.1], 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

LogReg => Train: DEKOIS; Test: DEKOIS; split: scaffold
No. of molecules in train set: 885, with 30 actives.
No. of molecules in test set: 336, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.864
- Train ROC-AUC:  	0.876
- Test ROC-AUC:   	0.956
- Best hyperparameters {'C': 0.001, 'penalty': 'l2', 'solver': 'lbfgs'}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.884 	> median: 0.816, mean: 0.812
> Test best conf. ROC-AUC: 0.965 	> median: 0.897, mean: 0.893
****************************************************************
CPU times: user 1.18 s, sys: 581 ms, total: 1.76 s
Wall time: 1.55 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: K-Neighbors Classifier</h3>
<b>DEKOIS  - Random and Stratified Scaffold Splitting</b>

In [20]:
%%time
from sklearn.neighbors import KNeighborsClassifier 

estimator_name = 'kNN'
estimator = KNeighborsClassifier()
hyperparams = {'n_neighbors': [25, 55, 125, 225], 
               'p': [1, 2]
             }

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

kNN => Train: DEKOIS; Test: DEKOIS; split: random
No. of molecules in train set: 915, with 30 actives.
No. of molecules in test set: 306, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.923
- Train ROC-AUC:  	0.944
- Test ROC-AUC:   	0.837
- Best hyperparameters {'n_neighbors': 55, 'p': 2}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.908 	> median: 0.841, mean: 0.833
> Test best conf. ROC-AUC: 0.917 	> median: 0.826, mean: 0.824
****************************************************************
CPU times: user 860 ms, sys: 88 ms, total: 948 ms
Wall time: 2.26 s


In [21]:
%%time
from sklearn.neighbors import KNeighborsClassifier 

estimator_name = 'kNN'
estimator = KNeighborsClassifier()
hyperparams = {'n_neighbors': [25, 55, 125, 225], 
               'p': [1, 2]
             }

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

kNN => Train: DEKOIS; Test: DEKOIS; split: scaffold
No. of molecules in train set: 885, with 30 actives.
No. of molecules in test set: 336, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.839
- Train ROC-AUC:  	0.871
- Test ROC-AUC:   	0.953
- Best hyperparameters {'n_neighbors': 225, 'p': 2}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.884 	> median: 0.816, mean: 0.812
> Test best conf. ROC-AUC: 0.965 	> median: 0.897, mean: 0.893
****************************************************************
CPU times: user 947 ms, sys: 85.4 ms, total: 1.03 s
Wall time: 1.5 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Decision Tree Classifier</h3>
<b>DEKOIS  - Random and Stratified Scaffold Splitting</b>

In [22]:
%%time
from sklearn.tree import DecisionTreeClassifier

estimator_name = 'DTree'
estimator = DecisionTreeClassifier(splitter='best')
hyperparams = {'criterion': ['gini', 'entropy'], 
               'max_depth': [2, 3, 5],
               'min_samples_split': [0.2, 0.25, 0.3],
               'max_features': [None, 'sqrt', 'log2']}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

DTree => Train: DEKOIS; Test: DEKOIS; split: random
No. of molecules in train set: 915, with 30 actives.
No. of molecules in test set: 306, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.861
- Train ROC-AUC:  	0.933
- Test ROC-AUC:   	0.826
- Best hyperparameters {'criterion': 'gini', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_split': 0.2}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.903 	> median: 0.827, mean: 0.827
> Test best conf. ROC-AUC: 0.929 	> median: 0.850, mean: 0.844
****************************************************************
CPU times: user 1.22 s, sys: 248 ms, total: 1.47 s
Wall time: 2.06 s


In [23]:
%%time
from sklearn.tree import DecisionTreeClassifier

estimator_name = 'DTree'
estimator = DecisionTreeClassifier(splitter='best')
hyperparams = {'criterion': ['gini', 'entropy'], 
               'max_depth': [2, 3, 5],
               'min_samples_split': [0.2, 0.25, 0.3],
               'max_features': [None, 'sqrt', 'log2']}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

DTree => Train: DEKOIS; Test: DEKOIS; split: scaffold
No. of molecules in train set: 885, with 30 actives.
No. of molecules in test set: 336, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.832
- Train ROC-AUC:  	0.869
- Test ROC-AUC:   	0.944
- Best hyperparameters {'criterion': 'entropy', 'max_depth': 3, 'max_features': 'sqrt', 'min_samples_split': 0.25}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.884 	> median: 0.816, mean: 0.812
> Test best conf. ROC-AUC: 0.965 	> median: 0.897, mean: 0.893
****************************************************************
CPU times: user 1.23 s, sys: 249 ms, total: 1.48 s
Wall time: 1.81 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Bagging Classifier (k-NN as base estimator) </h3>
<b>DEKOIS  - Random and Stratified Scaffold Splitting</b>

In [24]:
%%time
from sklearn.ensemble import BaggingClassifier

estimator_name = 'BagClf-kNN'
knn = KNeighborsClassifier(n_neighbors=125, p=1, 
                           weights='distance')
estimator = BaggingClassifier(base_estimator=knn, 
                              n_jobs=6, oob_score=True)

hyperparams = {'n_estimators': [300]}

# RANDOM Train test splitting
# split_and_gs(X, y, estimator, hyperparams, 
#              splitting='random', test_size=0.25, 
#              scaffold_series=None)

CPU times: user 15.5 ms, sys: 8.99 ms, total: 24.5 ms
Wall time: 285 ms


In [25]:
%%time
# from sklearn.ensemble import BaggingClassifier

estimator_name = 'BagClf-kNN'
knn = KNeighborsClassifier(n_neighbors=125, p=1, 
                           weights='distance')
estimator = BaggingClassifier(base_estimator=knn, 
                              n_jobs=6, oob_score=True)

hyperparams = {'n_estimators': [300]}

# SCAFFOLD Train test splitting
# split_and_gs(X, y, estimator, hyperparams, 
#              splitting='scaffold', test_size=0.25, 
#              scaffold_series=scaffold_series)

CPU times: user 42 µs, sys: 15 µs, total: 57 µs
Wall time: 65.6 µs


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS:Random Forest </h3>
<b>DEKOIS  - Random and Stratified Scaffold Splitting</b>

In [26]:
%%time
from sklearn.ensemble import RandomForestClassifier

estimator_name = 'RandForest'
estimator = RandomForestClassifier()
hyperparams = {'n_estimators': [300, 500],
             'max_depth': [2,  5],
             'min_samples_leaf': [0.1, 0.2],
             'max_features': ['log2', 'sqrt']
            }

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

RandForest => Train: DEKOIS; Test: DEKOIS; split: random
No. of molecules in train set: 915, with 30 actives.
No. of molecules in test set: 306, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.896
- Train ROC-AUC:  	0.918
- Test ROC-AUC:   	0.856
- Best hyperparameters {'max_depth': 2, 'max_features': 'log2', 'min_samples_leaf': 0.2, 'n_estimators': 500}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.904 	> median: 0.842, mean: 0.835
> Test best conf. ROC-AUC: 0.891 	> median: 0.818, mean: 0.819
****************************************************************
CPU times: user 2.03 s, sys: 277 ms, total: 2.31 s
Wall time: 21.3 s


In [27]:
%%time
from sklearn.ensemble import RandomForestClassifier

estimator_name = 'RandForest'
estimator = RandomForestClassifier()
hyperparams = {'n_estimators': [300, 500],
             'max_depth': [2,  5],
             'min_samples_leaf': [0.1, 0.2],
             'max_features': ['log2', 'sqrt']
            }

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams, 
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

RandForest => Train: DEKOIS; Test: DEKOIS; split: scaffold
No. of molecules in train set: 885, with 30 actives.
No. of molecules in test set: 336, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.855
- Train ROC-AUC:  	0.898
- Test ROC-AUC:   	0.949
- Best hyperparameters {'max_depth': 2, 'max_features': 'log2', 'min_samples_leaf': 0.2, 'n_estimators': 300}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.884 	> median: 0.816, mean: 0.812
> Test best conf. ROC-AUC: 0.965 	> median: 0.897, mean: 0.893
****************************************************************
CPU times: user 1.82 s, sys: 235 ms, total: 2.05 s
Wall time: 21.4 s


***
<h2 style='background-color: #F9E5AB; padding: 5px;'>
    Train-Test with DUD Library
</h2>
<div style='background-color: #FE8460; min-height: 5px'></div>

In [28]:
library = 'DUD'

# Train and test over DUDU
X = X_merged_dksc.loc[library]
y = y_true_merged.loc[library]
scaffold_series = df_scff_murcko['scff_generic'].loc[library]

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Linear SVM </h3>
<b>DUD  - Random and Stratified Scaffold Splitting</b>

In [29]:
%%time
from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-15, 1e-6, 4)}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

LinearSVC => Train: DUD; Test: DUD; split: random
No. of molecules in train set: 3669, with 106 actives.
No. of molecules in test set: 1224, with 35 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.764
- Train ROC-AUC:  	0.801
- Test ROC-AUC:   	0.744
- Best hyperparameters {'C': 1e-15}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.702 	> median: 0.544, mean: 0.542
> Test best conf. ROC-AUC: 0.804 	> median: 0.654, mean: 0.657
****************************************************************
CPU times: user 2.17 s, sys: 80.4 ms, total: 2.25 s
Wall time: 9.29 s


In [30]:
%%time
# from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-15, 1e-6, 4)}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

LinearSVC => Train: DUD; Test: DUD; split: scaffold
No. of molecules in train set: 3564, with 105 actives.
No. of molecules in test set: 1329, with 36 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.883
- Train ROC-AUC:  	0.902
- Test ROC-AUC:   	0.546
- Best hyperparameters {'C': 1e-06}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.747 	> median: 0.545, mean: 0.547
> Test best conf. ROC-AUC: 0.719 	> median: 0.640, mean: 0.638
****************************************************************
CPU times: user 1.62 s, sys: 37.7 ms, total: 1.66 s
Wall time: 8.12 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Radial Basis Function SVM </h3>
<b>DUD  - Random and Stratified Scaffold Splitting</b>

In [31]:
%%time

estimator_name = 'rbfSVC'
estimator = SVC(kernel = 'rbf', probability=True)
hyperparams = {'C': np.geomspace(1e-8, 1e-2, 4), 
               'gamma': np.geomspace(1e-6, 1e0, 4)}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

rbfSVC => Train: DUD; Test: DUD; split: random
No. of molecules in train set: 3669, with 106 actives.
No. of molecules in test set: 1224, with 35 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.857
- Train ROC-AUC:  	0.992
- Test ROC-AUC:   	0.953
- Best hyperparameters {'C': 0.01, 'gamma': 0.01}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.708 	> median: 0.574, mean: 0.576
> Test best conf. ROC-AUC: 0.778 	> median: 0.553, mean: 0.556
****************************************************************
CPU times: user 4.16 s, sys: 121 ms, total: 4.28 s
Wall time: 42.1 s


In [32]:
%%time

estimator_name = 'rbfSVC'
estimator = SVC(kernel = 'rbf', probability=True)
hyperparams = {'C': np.geomspace(1e-8, 1e-2, 4), 
               'gamma': np.geomspace(1e-6, 1e0, 4)}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

rbfSVC => Train: DUD; Test: DUD; split: scaffold
No. of molecules in train set: 3564, with 105 actives.
No. of molecules in test set: 1329, with 36 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.941
- Train ROC-AUC:  	0.997
- Test ROC-AUC:   	0.587
- Best hyperparameters {'C': 0.01, 'gamma': 0.01}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.747 	> median: 0.545, mean: 0.547
> Test best conf. ROC-AUC: 0.719 	> median: 0.640, mean: 0.638
****************************************************************
CPU times: user 3.69 s, sys: 148 ms, total: 3.84 s
Wall time: 42.9 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'> GS: Logistic Regression </h3>
<b>DUD  - Random and Stratified Scaffold Splitting</b>

In [33]:
%%time
from sklearn.linear_model import LogisticRegression

estimator_name = 'LogReg'
estimator = LogisticRegression()
hyperparams = {'C': [0.00001, 0.0001, 0.001, 0.01, 0.1], 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

LogReg => Train: DUD; Test: DUD; split: random
No. of molecules in train set: 3669, with 106 actives.
No. of molecules in test set: 1224, with 35 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.861
- Train ROC-AUC:  	0.922
- Test ROC-AUC:   	0.832
- Best hyperparameters {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.713 	> median: 0.558, mean: 0.560
> Test best conf. ROC-AUC: 0.814 	> median: 0.604, mean: 0.604
****************************************************************
CPU times: user 1.57 s, sys: 508 ms, total: 2.08 s
Wall time: 5.61 s


In [34]:
%%time
# from sklearn.linear_model import LogisticRegression

estimator_name = 'LogReg'
estimator = LogisticRegression()
hyperparams = {'C': [0.00001, 0.0001, 0.001, 0.01, 0.1], 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

LogReg => Train: DUD; Test: DUD; split: scaffold
No. of molecules in train set: 3564, with 105 actives.
No. of molecules in test set: 1329, with 36 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.890
- Train ROC-AUC:  	0.962
- Test ROC-AUC:   	0.618
- Best hyperparameters {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.747 	> median: 0.545, mean: 0.547
> Test best conf. ROC-AUC: 0.719 	> median: 0.640, mean: 0.638
****************************************************************
CPU times: user 1.55 s, sys: 441 ms, total: 1.99 s
Wall time: 5.14 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: K-Neighbors Classifier</h3>
<b>DUD  - Random and Stratified Scaffold Splitting</b>

In [35]:
%%time
from sklearn.neighbors import KNeighborsClassifier 

estimator_name = 'kNN'
estimator = KNeighborsClassifier()
hyperparams = {'n_neighbors': [25, 55, 125, 225], 
               'p': [1, 2]
             }

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

kNN => Train: DUD; Test: DUD; split: random
No. of molecules in train set: 3669, with 106 actives.
No. of molecules in test set: 1224, with 35 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.810
- Train ROC-AUC:  	0.883
- Test ROC-AUC:   	0.793
- Best hyperparameters {'n_neighbors': 125, 'p': 1}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.714 	> median: 0.563, mean: 0.564
> Test best conf. ROC-AUC: 0.764 	> median: 0.587, mean: 0.592
****************************************************************
CPU times: user 5.38 s, sys: 177 ms, total: 5.56 s
Wall time: 18.4 s


In [36]:
%%time
from sklearn.neighbors import KNeighborsClassifier 

estimator_name = 'kNN'
estimator = KNeighborsClassifier()
hyperparams = {'n_neighbors': [25, 55, 125, 225], 
               'p': [1, 2]
             }

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

kNN => Train: DUD; Test: DUD; split: scaffold
No. of molecules in train set: 3564, with 105 actives.
No. of molecules in test set: 1329, with 36 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.827
- Train ROC-AUC:  	0.951
- Test ROC-AUC:   	0.596
- Best hyperparameters {'n_neighbors': 55, 'p': 1}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.747 	> median: 0.545, mean: 0.547
> Test best conf. ROC-AUC: 0.719 	> median: 0.640, mean: 0.638
****************************************************************
CPU times: user 5.35 s, sys: 92.3 ms, total: 5.44 s
Wall time: 17 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Decision Tree Classifier</h3>
<b>DUD  - Random and Stratified Scaffold Splitting</b>

In [37]:
%%time
from sklearn.tree import DecisionTreeClassifier

estimator_name = 'DTree'
estimator = DecisionTreeClassifier(splitter='best')
hyperparams = {'criterion': ['gini', 'entropy'], 
               'max_depth': [2, 3, 5],
               'min_samples_split': [0.2, 0.25, 0.3],
               'max_features': [None, 'sqrt', 'log2']}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

DTree => Train: DUD; Test: DUD; split: random
No. of molecules in train set: 3669, with 106 actives.
No. of molecules in test set: 1224, with 35 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.788
- Train ROC-AUC:  	0.842
- Test ROC-AUC:   	0.765
- Best hyperparameters {'criterion': 'gini', 'max_depth': 5, 'max_features': None, 'min_samples_split': 0.2}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.720 	> median: 0.572, mean: 0.573
> Test best conf. ROC-AUC: 0.721 	> median: 0.565, mean: 0.563
****************************************************************
CPU times: user 1.57 s, sys: 157 ms, total: 1.73 s
Wall time: 3.18 s


In [38]:
%%time
from sklearn.tree import DecisionTreeClassifier

estimator_name = 'DTree'
estimator = DecisionTreeClassifier(splitter='best')
hyperparams = {'criterion': ['gini', 'entropy'], 
               'max_depth': [2, 3, 5],
               'min_samples_split': [0.2, 0.25, 0.3],
               'max_features': [None, 'sqrt', 'log2']}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

DTree => Train: DUD; Test: DUD; split: scaffold
No. of molecules in train set: 3564, with 105 actives.
No. of molecules in test set: 1329, with 36 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.825
- Train ROC-AUC:  	0.875
- Test ROC-AUC:   	0.578
- Best hyperparameters {'criterion': 'gini', 'max_depth': 5, 'max_features': None, 'min_samples_split': 0.3}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.747 	> median: 0.545, mean: 0.547
> Test best conf. ROC-AUC: 0.719 	> median: 0.640, mean: 0.638
****************************************************************
CPU times: user 1.73 s, sys: 122 ms, total: 1.85 s
Wall time: 3.19 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Bagging Classifier (k-NN as base estimator) </h3>
<b>DUD  - Random and Stratified Scaffold Splitting</b>

In [39]:
%%time
from sklearn.ensemble import BaggingClassifier

estimator_name = 'BagClf-kNN'
knn = KNeighborsClassifier(n_neighbors=125, p=1, 
                           weights='distance')
estimator = BaggingClassifier(base_estimator=knn, 
                              n_jobs=6, oob_score=True)

hyperparams = {'n_estimators': [300]}

# RANDOM Train test splitting
# split_and_gs(X, y, estimator, hyperparams, 
#              splitting='random', test_size=0.25, 
#              scaffold_series=None)

CPU times: user 48 µs, sys: 7 µs, total: 55 µs
Wall time: 62.9 µs


In [40]:
%%time
# from sklearn.ensemble import BaggingClassifier

estimator_name = 'BagClf-kNN'
knn = KNeighborsClassifier(n_neighbors=125, p=1, 
                           weights='distance')
estimator = BaggingClassifier(base_estimator=knn, 
                              n_jobs=6, oob_score=True)

hyperparams = {'n_estimators': [300]}

# SCAFFOLD Train test splitting
# split_and_gs(X, y, estimator, hyperparams, 
#              splitting='scaffold', test_size=0.25, 
#              scaffold_series=scaffold_series)

CPU times: user 35 µs, sys: 6 µs, total: 41 µs
Wall time: 47.7 µs


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS:Random Forest </h3>
<b>DUD  - Random and Stratified Scaffold Splitting</b>

In [41]:
%%time
from sklearn.ensemble import RandomForestClassifier

estimator_name = 'RandForest'
estimator = RandomForestClassifier()
hyperparams = {'n_estimators': [300, 500],
             'max_depth': [2,  5],
             'min_samples_leaf': [0.1, 0.2],
             'max_features': ['log2', 'sqrt']
            }

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

RandForest => Train: DUD; Test: DUD; split: random
No. of molecules in train set: 3669, with 106 actives.
No. of molecules in test set: 1224, with 35 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.764
- Train ROC-AUC:  	0.823
- Test ROC-AUC:   	0.778
- Best hyperparameters {'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 0.1, 'n_estimators': 300}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.735 	> median: 0.583, mean: 0.583
> Test best conf. ROC-AUC: 0.687 	> median: 0.537, mean: 0.535
****************************************************************
CPU times: user 2.6 s, sys: 99.3 ms, total: 2.7 s
Wall time: 35.6 s


In [42]:
%%time
from sklearn.ensemble import RandomForestClassifier

estimator_name = 'RandForest'
estimator = RandomForestClassifier()
hyperparams = {'n_estimators': [300, 500],
             'max_depth': [2,  5],
             'min_samples_leaf': [0.1, 0.2],
             'max_features': ['log2', 'sqrt']
            }

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

RandForest => Train: DUD; Test: DUD; split: scaffold
No. of molecules in train set: 3564, with 105 actives.
No. of molecules in test set: 1329, with 36 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.812
- Train ROC-AUC:  	0.892
- Test ROC-AUC:   	0.642
- Best hyperparameters {'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 0.1, 'n_estimators': 300}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.747 	> median: 0.545, mean: 0.547
> Test best conf. ROC-AUC: 0.719 	> median: 0.640, mean: 0.638
****************************************************************
CPU times: user 2.54 s, sys: 65.2 ms, total: 2.61 s
Wall time: 32.7 s


***

<h2 style='background-color: #F9E5AB; padding: 5px;'>
    Train-Test with different libraries 
</h2>
<div style='background-color: #FE8460; min-height: 5px'></div>

##### Here, Random or Scaffold Splitting are not used

In [43]:
# Train DEKOIS, test DUD
@capture_GS_results(results_dict)
def split_by_library_and_gs(train_name, test_name, estimator_name,
                            X, y, lib_train_name, lib_test_name, **kwargs):
    '''Train-Test "split" by library, and run Grid Search. Splits the main dataframe by library
    using different molecular libraries for Training and Testing.'''
    splitting='by_library'
    
    X_train = X.loc[lib_train_name]
    y_train = y.loc[lib_train_name]

    X_test = X.loc[lib_test_name]
    y_test = y.loc[lib_test_name]

    # Verbose
    print(f'{estimator_name} => Train: {train_name}; Test: {test_name}; split: {splitting}')
    
    # Function to run Grid Search
    #----------------------------
    gs_results = run_grid_search(estimator, 
                    X_train, y_train, X_test, y_test, 
                    hyperparams = hyperparams,  **kwargs)

    # Function to extract ROC results from DkSc values 
    #-------------------------------------------------
    dksc_results = get_roc_auc_DkSc(X_train, y_train, X_test, y_test)
    
    # Return both list of results to captured by the decorador function
    return [train_name, test_name, estimator_name, splitting] + gs_results + dksc_results

In [44]:
# **********************************************
# Train and Test sets from diferent libraries
# **********************************************
X = X_merged_dksc
y = y_true_merged

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Linear SVM  </h3>
<b>Train and Test with different libraries</b>

In [45]:
%%time
from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-15, 1e-6, 4)}

# Train DEKOIS, test DUD
train_lib = 'DEKOIS'
test_lib = 'DUD'
split_by_library_and_gs(
    train_lib, test_lib, estimator_name,
    X, y, train_lib, test_lib)

LinearSVC => Train: DEKOIS; Test: DUD; split: by_library
No. of molecules in train set: 1221, with 40 actives.
No. of molecules in test set: 4893, with 141 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.889
- Train ROC-AUC:  	0.894
- Test ROC-AUC:   	0.607
- Best hyperparameters {'C': 1e-15}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.894 	> median: 0.833, mean: 0.831
> Test best conf. ROC-AUC: 0.720 	> median: 0.571, mean: 0.571
****************************************************************
CPU times: user 818 ms, sys: 15.9 ms, total: 834 ms
Wall time: 1.31 s


In [46]:
%%time
# from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-15, 1e-6, 4)}

# Train DEKOIS, test DUD
train_lib = 'DUD'
test_lib = 'DEKOIS'
split_by_library_and_gs(
    train_lib, test_lib, estimator_name,
    X, y, train_lib, test_lib)

LinearSVC => Train: DUD; Test: DEKOIS; split: by_library
No. of molecules in train set: 4893, with 141 actives.
No. of molecules in test set: 1221, with 40 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.782
- Train ROC-AUC:  	0.800
- Test ROC-AUC:   	0.870
- Best hyperparameters {'C': 1e-15}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.720 	> median: 0.571, mean: 0.571
> Test best conf. ROC-AUC: 0.894 	> median: 0.833, mean: 0.831
****************************************************************
CPU times: user 2.39 s, sys: 78.3 ms, total: 2.46 s
Wall time: 11.3 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Radial Basis Function SVM </h3>
<b>Train and Test with different libraries</b>

In [47]:
%%time
estimator = SVC(kernel = 'rbf', probability=True)

estimator_name = 'rbfSVC'
hyperparams = {'C': np.geomspace(1e-8, 1e-2, 4), 
               'gamma': np.geomspace(1e-6, 1e0, 4)}

# Train DEKOIS, test DUD
train_lib = 'DEKOIS'
test_lib = 'DUD'
split_by_library_and_gs(
    train_lib, test_lib, estimator_name,
    X, y, train_lib, test_lib)

rbfSVC => Train: DEKOIS; Test: DUD; split: by_library
No. of molecules in train set: 1221, with 40 actives.
No. of molecules in test set: 4893, with 141 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.888
- Train ROC-AUC:  	0.893
- Test ROC-AUC:   	0.613
- Best hyperparameters {'C': 1e-08, 'gamma': 1e-06}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.894 	> median: 0.833, mean: 0.831
> Test best conf. ROC-AUC: 0.720 	> median: 0.571, mean: 0.571
****************************************************************
CPU times: user 1.14 s, sys: 81.5 ms, total: 1.22 s
Wall time: 4.13 s


In [48]:
%%time
estimator = SVC(kernel = 'rbf', probability=True)

estimator_name = 'rbfSVC'
hyperparams = {'C': np.geomspace(1e-8, 1e-2, 4), 
               'gamma': np.geomspace(1e-6, 1e0, 4)}

# Train DEKOIS, test DUD
train_lib = 'DUD'
test_lib = 'DEKOIS'
split_by_library_and_gs(
    train_lib, test_lib, estimator_name,
    X, y, train_lib, test_lib)

rbfSVC => Train: DUD; Test: DEKOIS; split: by_library
No. of molecules in train set: 4893, with 141 actives.
No. of molecules in test set: 1221, with 40 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.867
- Train ROC-AUC:  	0.977
- Test ROC-AUC:   	0.638
- Best hyperparameters {'C': 0.01, 'gamma': 0.01}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.720 	> median: 0.571, mean: 0.571
> Test best conf. ROC-AUC: 0.894 	> median: 0.833, mean: 0.831
****************************************************************
CPU times: user 5.83 s, sys: 186 ms, total: 6.01 s
Wall time: 1min 10s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Logistic Regression</h3>
<b>Train and Test with different libraries</b>

In [49]:
%%time
from sklearn.linear_model import LogisticRegression

estimator_name = 'LogReg'
estimator = LogisticRegression()
hyperparams = {'C': np.geomspace(1e-4, 1e5, 4),
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# Train DEKOIS, test DUD
train_lib = 'DEKOIS'
test_lib = 'DUD'
split_by_library_and_gs(
    train_lib, test_lib, estimator_name,
    X, y, train_lib, test_lib)

LogReg => Train: DEKOIS; Test: DUD; split: by_library
No. of molecules in train set: 1221, with 40 actives.
No. of molecules in test set: 4893, with 141 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.891
- Train ROC-AUC:  	0.890
- Test ROC-AUC:   	0.603
- Best hyperparameters {'C': 0.0001, 'penalty': 'l2', 'solver': 'lbfgs'}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.894 	> median: 0.833, mean: 0.831
> Test best conf. ROC-AUC: 0.720 	> median: 0.571, mean: 0.571
****************************************************************
CPU times: user 1.23 s, sys: 435 ms, total: 1.66 s
Wall time: 21.8 s


In [50]:
%%time
# from sklearn.linear_model import LogisticRegression

estimator_name = 'LogReg'
estimator = LogisticRegression()
hyperparams = {'C': np.geomspace(1e-4, 1e5, 4), 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# Train DEKOIS, test DUD
train_lib = 'DUD'
test_lib = 'DEKOIS'
split_by_library_and_gs(
    train_lib, test_lib, estimator_name,
    X, y, train_lib, test_lib)

LogReg => Train: DUD; Test: DEKOIS; split: by_library
No. of molecules in train set: 4893, with 141 actives.
No. of molecules in test set: 1221, with 40 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.815
- Train ROC-AUC:  	0.922
- Test ROC-AUC:   	0.758
- Best hyperparameters {'C': 100.0, 'penalty': 'l2', 'solver': 'liblinear'}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.720 	> median: 0.571, mean: 0.571
> Test best conf. ROC-AUC: 0.894 	> median: 0.833, mean: 0.831
****************************************************************
CPU times: user 1.98 s, sys: 480 ms, total: 2.46 s
Wall time: 45.5 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS:Random Forest  </h3>
<b>Train and Test with different libraries</b>

In [51]:
%%time
from sklearn.ensemble import RandomForestClassifier

estimator_name = 'RandForest'
estimator = RandomForestClassifier()
hyperparams = {'n_estimators': [300, 500],
             'max_depth': [3,  5, 7],
             'min_samples_leaf': [0.03, 0.05],
               'min_samples_split': [0.2,  0.3],
             'max_features': ['sqrt']
            }

# Train DEKOIS, test DUD
train_lib = 'DEKOIS'
test_lib = 'DUD'
split_by_library_and_gs(
    train_lib, test_lib, estimator_name,
    X, y, train_lib, test_lib)

RandForest => Train: DEKOIS; Test: DUD; split: by_library
No. of molecules in train set: 1221, with 40 actives.
No. of molecules in test set: 4893, with 141 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.883
- Train ROC-AUC:  	0.930
- Test ROC-AUC:   	0.584
- Best hyperparameters {'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 0.03, 'min_samples_split': 0.3, 'n_estimators': 300}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.894 	> median: 0.833, mean: 0.831
> Test best conf. ROC-AUC: 0.720 	> median: 0.571, mean: 0.571
****************************************************************
CPU times: user 2.3 s, sys: 91.9 ms, total: 2.39 s
Wall time: 42 s


In [52]:
%%time
from sklearn.ensemble import RandomForestClassifier

estimator_name = 'RandForest'
estimator = RandomForestClassifier()
hyperparams = {'n_estimators': [300, 500],
             'max_depth': [3,  5, 7],
             'min_samples_leaf': [0.03, 0.05],
               'min_samples_split': [0.2,  0.3],
             'max_features': ['sqrt']
            }

# Train DEKOIS, test DUD
train_lib = 'DUD'
test_lib = 'DEKOIS'
split_by_library_and_gs(
    train_lib, test_lib, estimator_name,
    X, y, train_lib, test_lib)

RandForest => Train: DUD; Test: DEKOIS; split: by_library
No. of molecules in train set: 4893, with 141 actives.
No. of molecules in test set: 1221, with 40 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.802
- Train ROC-AUC:  	0.866
- Test ROC-AUC:   	0.675
- Best hyperparameters {'max_depth': 7, 'max_features': 'sqrt', 'min_samples_leaf': 0.03, 'min_samples_split': 0.2, 'n_estimators': 500}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.720 	> median: 0.571, mean: 0.571
> Test best conf. ROC-AUC: 0.894 	> median: 0.833, mean: 0.831
****************************************************************
CPU times: user 6.2 s, sys: 182 ms, total: 6.38 s
Wall time: 1min 31s


***
<h2 style='background-color: #F9E5AB; padding: 5px;'>
    Merged libraries 
</h2>
<div style='background-color: #FE8460; min-height: 5px'></div>

#### DEKOIS, DUD and COCRYS are  treated as one unique library

In [53]:
# Train and test over 
X = X_merged_dksc
y = y_true_merged
library = 'Merged'
scaffold_series = df_scff_murcko['scff_generic']

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Linear SVM </h3>
<b>Merged Libraries</b>

In [54]:
%%time
from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-15, 1e-9, 3)}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

LinearSVC => Train: Merged; Test: Merged; split: random
No. of molecules in train set: 4674, with 225 actives.
No. of molecules in test set: 1559, with 75 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.723
- Train ROC-AUC:  	0.815
- Test ROC-AUC:   	0.745
- Best hyperparameters {'C': 1e-12}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.696 	> median: 0.629, mean: 0.624
> Test best conf. ROC-AUC: 0.648 	> median: 0.551, mean: 0.555
****************************************************************
CPU times: user 3.01 s, sys: 77.1 ms, total: 3.09 s
Wall time: 15.5 s


In [55]:
%%time
# from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-15, 1e-9, 3)}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

LinearSVC => Train: Merged; Test: Merged; split: scaffold
No. of molecules in train set: 4449, with 225 actives.
No. of molecules in test set: 1784, with 75 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.695
- Train ROC-AUC:  	0.685
- Test ROC-AUC:   	0.688
- Best hyperparameters {'C': 1e-12}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.686 	> median: 0.593, mean: 0.590
> Test best conf. ROC-AUC: 0.716 	> median: 0.650, mean: 0.645
****************************************************************
CPU times: user 3.02 s, sys: 38.2 ms, total: 3.06 s
Wall time: 12.5 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Radial Basis Function SVM</h3>
<b>Merged Libraries</b>

In [56]:
%%time
from sklearn.svm import SVC

estimator_name = 'rbfSVC'
estimator = SVC(kernel = 'rbf', probability=True)
hyperparams = {'C': np.geomspace(1e0, 1e2, 3), 
               'gamma': np.geomspace(1e-4, 1e0, 3)}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

rbfSVC => Train: Merged; Test: Merged; split: random
No. of molecules in train set: 4674, with 225 actives.
No. of molecules in test set: 1559, with 75 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.858
- Train ROC-AUC:  	1.000
- Test ROC-AUC:   	0.912
- Best hyperparameters {'C': 10.0, 'gamma': 0.01}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.673 	> median: 0.599, mean: 0.599
> Test best conf. ROC-AUC: 0.718 	> median: 0.633, mean: 0.631
****************************************************************
CPU times: user 9.33 s, sys: 183 ms, total: 9.51 s
Wall time: 3min 30s


In [57]:
%%time

estimator_name = 'rbfSVC'
estimator = SVC(kernel = 'rbf', probability=True)
hyperparams = {'C': np.geomspace(1e0, 1e2, 3), 
               'gamma': np.geomspace(1e-4, 1e0, 3)}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

rbfSVC => Train: Merged; Test: Merged; split: scaffold
No. of molecules in train set: 4449, with 225 actives.
No. of molecules in test set: 1784, with 75 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.854
- Train ROC-AUC:  	1.000
- Test ROC-AUC:   	0.773
- Best hyperparameters {'C': 10.0, 'gamma': 0.01}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.686 	> median: 0.593, mean: 0.590
> Test best conf. ROC-AUC: 0.716 	> median: 0.650, mean: 0.645
****************************************************************
CPU times: user 7.83 s, sys: 196 ms, total: 8.03 s
Wall time: 3min 5s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Logistic Regression</h3>
<b>Merged Libraries</b>

In [58]:
%%time
from sklearn.linear_model import LogisticRegression

estimator_name = 'LogReg'
estimator = LogisticRegression()
hyperparams = {'C': np.geomspace(1e-4, 1e4, 3), 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

LogReg => Train: Merged; Test: Merged; split: random
No. of molecules in train set: 4674, with 225 actives.
No. of molecules in test set: 1559, with 75 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.824
- Train ROC-AUC:  	0.885
- Test ROC-AUC:   	0.852
- Best hyperparameters {'C': 1.0, 'penalty': 'l1', 'solver': 'liblinear'}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.682 	> median: 0.608, mean: 0.605
> Test best conf. ROC-AUC: 0.707 	> median: 0.614, mean: 0.612
****************************************************************
CPU times: user 7.14 s, sys: 510 ms, total: 7.65 s
Wall time: 35.7 s


In [59]:
%%time
# from sklearn.linear_model import LogisticRegression

estimator_name = 'LogReg'
estimator = LogisticRegression()
hyperparams = {'C': np.geomspace(1e-4, 1e5, 4), 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

LogReg => Train: Merged; Test: Merged; split: scaffold
No. of molecules in train set: 4449, with 225 actives.
No. of molecules in test set: 1784, with 75 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.817
- Train ROC-AUC:  	0.905
- Test ROC-AUC:   	0.746
- Best hyperparameters {'C': 100.0, 'penalty': 'l1', 'solver': 'liblinear'}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.686 	> median: 0.593, mean: 0.590
> Test best conf. ROC-AUC: 0.716 	> median: 0.650, mean: 0.645
****************************************************************
CPU times: user 9.66 s, sys: 608 ms, total: 10.3 s
Wall time: 51.2 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Decision Tree Classifier</h3>
<b>Merged Libraries</b>

In [60]:
%%time
from sklearn.tree import DecisionTreeClassifier

estimator_name = 'DTree'
estimator = DecisionTreeClassifier(splitter='best')
hyperparams = {'criterion': ['gini', 'entropy'], 
               'max_depth': [2, 3, 5],
               'min_samples_split': [0.2,  0.3],
               'min_samples_leaf': [0.02, 0.05, 0.1],
               'max_features': [None, 'sqrt', 'log2']}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

DTree => Train: Merged; Test: Merged; split: random
No. of molecules in train set: 4674, with 225 actives.
No. of molecules in test set: 1559, with 75 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.752
- Train ROC-AUC:  	0.790
- Test ROC-AUC:   	0.737
- Best hyperparameters {'criterion': 'entropy', 'max_depth': 5, 'max_features': None, 'min_samples_leaf': 0.05, 'min_samples_split': 0.2}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.674 	> median: 0.599, mean: 0.597
> Test best conf. ROC-AUC: 0.729 	> median: 0.636, mean: 0.637
****************************************************************
CPU times: user 2.85 s, sys: 335 ms, total: 3.18 s
Wall time: 6.7 s


In [61]:
%%time
from sklearn.tree import DecisionTreeClassifier

estimator_name = 'DTree'
estimator = DecisionTreeClassifier(splitter='best')
hyperparams = {'criterion': ['gini', 'entropy'], 
               'max_depth': [2, 3, 5],
               'min_samples_split': [0.2,  0.3],
               'min_samples_leaf': [0.02, 0.05, 0.1],
               'max_features': [None, 'sqrt', 'log2']}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

DTree => Train: Merged; Test: Merged; split: scaffold
No. of molecules in train set: 4449, with 225 actives.
No. of molecules in test set: 1784, with 75 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.727
- Train ROC-AUC:  	0.798
- Test ROC-AUC:   	0.653
- Best hyperparameters {'criterion': 'gini', 'max_depth': 5, 'max_features': None, 'min_samples_leaf': 0.05, 'min_samples_split': 0.3}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.686 	> median: 0.593, mean: 0.590
> Test best conf. ROC-AUC: 0.716 	> median: 0.650, mean: 0.645
****************************************************************
CPU times: user 2.62 s, sys: 323 ms, total: 2.95 s
Wall time: 6.45 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Bagging Classifier (k-NN as base estimator) </h3>
<b>Merged Libraries</b>

In [62]:
%%time
from sklearn.ensemble import BaggingClassifier

estimator_name = 'kNN'
knn = KNeighborsClassifier(n_neighbors=125, p=1, 
                           weights='distance')
estimator = BaggingClassifier(base_estimator=knn, 
                              n_jobs=3, oob_score=True)

hyperparams = {'n_estimators': [300]}

# RANDOM Train test splitting
# split_and_gs(X, y, estimator, hyperparams, 
#              splitting='random', test_size=0.25, 
#              scaffold_series=None)

CPU times: user 58 µs, sys: 5 µs, total: 63 µs
Wall time: 72.2 µs


In [63]:
%%time
# from sklearn.ensemble import BaggingClassifier

estimator_name = 'kNN'
knn = KNeighborsClassifier(n_neighbors=125, p=1, 
                           weights='distance')
estimator = BaggingClassifier(base_estimator=knn, 
                              n_jobs=3, oob_score=True)

hyperparams = {'n_estimators': [300]}

# SCAFFOLD Train test splitting
# split_and_gs(X, y, estimator, hyperparams, 
#              splitting='scaffold', test_size=0.25, 
#              scaffold_series=scaffold_series)

CPU times: user 51 µs, sys: 5 µs, total: 56 µs
Wall time: 64.6 µs


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Random Forest </h3>
<b>Merged Libraries</b>

In [64]:
%%time
from sklearn.ensemble import RandomForestClassifier

estimator_name = 'RandForest'
estimator = RandomForestClassifier()
hyperparams = {'n_estimators': [300, 400],
               'max_depth': [2, 3],
               'min_samples_split': [0.1,  0.3],
               'min_samples_leaf': [0.02, 0.05],
               'max_features': ['sqrt']
            }

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

RandForest => Train: Merged; Test: Merged; split: random
No. of molecules in train set: 4674, with 225 actives.
No. of molecules in test set: 1559, with 75 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.733
- Train ROC-AUC:  	0.775
- Test ROC-AUC:   	0.757
- Best hyperparameters {'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 0.05, 'min_samples_split': 0.1, 'n_estimators': 300}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.693 	> median: 0.610, mean: 0.610
> Test best conf. ROC-AUC: 0.686 	> median: 0.598, mean: 0.598
****************************************************************
CPU times: user 3.29 s, sys: 107 ms, total: 3.4 s
Wall time: 41.7 s


In [65]:
%%time
from sklearn.ensemble import RandomForestClassifier

estimator_name = 'RandForest'
estimator = RandomForestClassifier()
hyperparams = {'n_estimators': [300, 400],
               'max_depth': [2, 3],
               'min_samples_split': [0.1,  0.3],
               'min_samples_leaf': [0.02, 0.05],
               'max_features': ['sqrt']
            }

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

RandForest => Train: Merged; Test: Merged; split: scaffold
No. of molecules in train set: 4449, with 225 actives.
No. of molecules in test set: 1784, with 75 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.736
- Train ROC-AUC:  	0.805
- Test ROC-AUC:   	0.660
- Best hyperparameters {'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 0.02, 'min_samples_split': 0.1, 'n_estimators': 400}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.686 	> median: 0.593, mean: 0.590
> Test best conf. ROC-AUC: 0.716 	> median: 0.650, mean: 0.645
****************************************************************
CPU times: user 3.91 s, sys: 111 ms, total: 4.02 s
Wall time: 40.7 s


## Get, Format, and Save the Results

In [66]:
### Save Results
row_names = ['Train_set', 'Test_set', 'Model name', 'Split', 
             'N_actives_train', 'N_actives_test', 'N_mols_train', 'Num_mols_test',
            'Mean-CV-ROC', 'ROC-AUC_train', 'ROC-AUC_test', 'best_params',
            'DkS_max_ROC_train',  'DkSc_med_ROC_train', 'DkSc_mean_ROC_train', 
            'DkS_max_ROC_test',  'DkSc_med_ROC_test', 'DkSc_mean_ROC_test']
results_df = pd.DataFrame(results_dict, index=row_names).T.reset_index().drop('index', axis=1)
results_df.to_csv('./GRID_SEARCH_CV_results.csv')
results_df

,Train_set,Test_set,Model name,Split,N_actives_train,N_actives_test,N_mols_train,Num_mols_test,Mean-CV-ROC,ROC-AUC_train,ROC-AUC_test,best_params,DkS_max_ROC_train,DkSc_med_ROC_train,DkSc_mean_ROC_train,DkS_max_ROC_test,DkSc_med_ROC_test,DkSc_mean_ROC_test
0,DEKOIS,DEKOIS,LinearSVC,random,915,30,306,10,0.82919,1,0.885811,{'C': 1},0.923183,0.850979,0.847989,0.877365,0.785304,0.780507
1,DEKOIS,DEKOIS,LinearSVC,scaffold,885,30,336,10,0.847271,0.874522,0.956442,{'C': 1e-15},0.883938,0.816355,0.81151,0.965491,0.896933,0.892571
2,DEKOIS,DEKOIS,rbfSVC,random,915,30,306,10,0.87307,0.87484,0.942905,"{'C': 1e-08, 'gamma': 1e-06}",0.88791,0.817834,0.814639,0.944426,0.88049,0.877574
3,DEKOIS,DEKOIS,rbfSVC,scaffold,885,30,336,10,0.846394,0.875127,0.952761,"{'C': 1e-08, 'gamma': 1e-06}",0.883938,0.816355,0.81151,0.965491,0.896933,0.892571
4,DEKOIS,DEKOIS,LogReg,random,915,30,306,10,0.922222,0.950132,0.805068,"{'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}",0.92904,0.866582,0.859682,0.857264,0.747213,0.744238
5,DEKOIS,DEKOIS,LogReg,scaffold,885,30,336,10,0.863548,0.875634,0.955828,"{'C': 0.001, 'penalty': 'l2', 'solver': 'lbfgs'}",0.883938,0.816355,0.81151,0.965491,0.896933,0.892571
6,DEKOIS,DEKOIS,kNN,random,915,30,306,10,0.922881,0.943955,0.836993,"{'n_neighbors': 55, 'p': 2}",0.907665,0.84064,0.833498,0.91723,0.825845,0.824364
7,DEKOIS,DEKOIS,kNN,scaffold,885,30,336,10,0.839474,0.870643,0.953374,"{'n_neighbors': 225, 'p': 2}",0.883938,0.816355,0.81151,0.965491,0.896933,0.892571
8,DEKOIS,DEKOIS,DTree,random,915,30,306,10,0.860546,0.933202,0.825676,"{'criterion': 'gini', 'max_depth': 5, 'max_fea...",0.903051,0.827053,0.826696,0.929054,0.849831,0.843931
9,DEKOIS,DEKOIS,DTree,scaffold,885,30,336,10,0.831969,0.868713,0.944479,"{'criterion': 'entropy', 'max_depth': 3, 'max_...",0.883938,0.816355,0.81151,0.965491,0.896933,0.892571


***
<h2 style='background-color: #F9E5AB; padding: 5px;'>
    Merged libraries: Random y target values in the train set
</h2>
<div style='background-color: #FE8460; min-height: 5px'></div>

#### DEKOIS, DUD and COCRYS are  treated as one unique library

In [67]:
# Train and test over 
X = X_merged_dksc
# ***** Permutate y values *****
y = y_true_merged.sample(frac=1)

library = 'Merged'
scaffold_series = df_scff_murcko['scff_generic']


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Linear SVM </h3>
<b>Merged Libraries</b>

In [68]:
%%time
from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-15, 1e-9, 3)}

# RANDOM Train test splitting
# split_and_gs(library, library, estimator_name,
#              X, y, estimator, hyperparams,
#              splitting='random', test_size=0.25, 
#              scaffold_series=None)

CPU times: user 303 µs, sys: 27 µs, total: 330 µs
Wall time: 287 µs


In [69]:
%%time
# from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-15, 1e-9, 3)}

# SCAFFOLD Train test splitting
# split_and_gs(library, library, estimator_name,
#              X, y, estimator, hyperparams,
#              splitting='scaffold', test_size=0.25, 
#              scaffold_series=scaffold_series)

CPU times: user 325 µs, sys: 29 µs, total: 354 µs
Wall time: 312 µs


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Radial Basis Function SVM</h3>
<b>Merged Libraries</b>

In [70]:
%%time
from sklearn.svm import SVC

estimator_name = 'rbfSVC'
estimator = SVC(kernel = 'rbf', probability=True)
hyperparams = {'C': np.geomspace(1e0, 1e2, 3), 
               'gamma': np.geomspace(1e-4, 1e0, 3)}

# RANDOM Train test splitting
# split_and_gs(library, library, estimator_name,
#              X, y, estimator, hyperparams,
#              splitting='random', test_size=0.25, 
#              scaffold_series=None)

CPU times: user 391 µs, sys: 36 µs, total: 427 µs
Wall time: 379 µs


In [71]:
%%time

estimator_name = 'rbfSVC'
estimator = SVC(kernel = 'rbf', probability=True)
hyperparams = {'C': np.geomspace(1e0, 1e2, 3), 
               'gamma': np.geomspace(1e-4, 1e0, 3)}

# SCAFFOLD Train test splitting
# split_and_gs(library, library, estimator_name,
#              X, y, estimator, hyperparams,
#              splitting='scaffold', test_size=0.25, 
#              scaffold_series=scaffold_series)

CPU times: user 411 µs, sys: 37 µs, total: 448 µs
Wall time: 381 µs


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Logistic Regression</h3>
<b>Merged Libraries</b>

In [72]:
%%time
from sklearn.linear_model import LogisticRegression

estimator_name = 'LogReg'
estimator = LogisticRegression()
hyperparams = {'C': np.geomspace(1e-4, 1e4, 3), 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# RANDOM Train test splitting
# split_and_gs(library, library, estimator_name,
#              X, y, estimator, hyperparams,
#              splitting='random', test_size=0.25, 
#              scaffold_series=None)

CPU times: user 349 µs, sys: 0 ns, total: 349 µs
Wall time: 313 µs


In [73]:
%%time
# from sklearn.linear_model import LogisticRegression

estimator_name = 'LogReg'
estimator = LogisticRegression()
hyperparams = {'C': np.geomspace(1e-4, 1e5, 4), 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# SCAFFOLD Train test splitting
# split_and_gs(library, library, estimator_name,
#              X, y, estimator, hyperparams,
#              splitting='scaffold', test_size=0.25, 
#              scaffold_series=scaffold_series)

CPU times: user 302 µs, sys: 27 µs, total: 329 µs
Wall time: 294 µs
